In [6]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [7]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '4'
os.environ['NUMEXPR_NUM_THREADS'] = '2'
import numexpr as ne

In [8]:
import os
os.environ["OPENAI_API_KEY"] = "your key"
import openai
openai.api_key = "your key"

In [9]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
andrew_gina_docs = SimpleDirectoryReader(input_files=["./assets/AndrewHuberman/sleep/115_Dr_Gina_Poe_Use_Sleep_to_Enhance_Learning_Memory_&_Emotional_State_Huberman_Lab_Podcast.txt"], filename_as_id=True).load_data()
vector_index = VectorStoreIndex.from_documents(andrew_gina_docs)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [10]:
vector_engine = vector_index.as_chat_engine()

In [17]:
from llama_index.core import ListIndex, ServiceContext

service_context = ServiceContext.from_defaults(chunk_size=1024)
nodes = service_context.node_parser.get_nodes_from_documents(andrew_gina_docs)
list_index = ListIndex(nodes)

C:\Users\Boris\AppData\Local\Temp\ipykernel_21124\3825651489.py:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(chunk_size=1024)


In [18]:
list_query_engine = list_index.as_query_engine(
    response_mode = "tree_summarize", use_async = True
)

In [19]:
from llama_index.core.tools.query_engine import QueryEngineTool

list_tool = QueryEngineTool.from_defaults(
    query_engine = list_query_engine,
    description="Useful for summarisation of the podcast about sleep and memory with dr. Gina Poe"
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_engine,
    description="Useful fro retrieving of specific content about sleep and memory in the podcast topic"
)

In [24]:
from llama_index.core.selectors.pydantic_selectors import PydanticSingleSelector
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine

query_engine = RouterQueryEngine(
    selector=PydanticSingleSelector.from_defaults(),
    query_engine_tools=[
        list_tool,
        vector_tool
    ]
)

In [25]:
from llama_index.core.selectors.llm_selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        list_tool,
        vector_tool
    ] 
)

In [26]:
openai.log = "debug"

In [29]:
import nest_asyncio
nest_asyncio.apply()

In [30]:
from llama_index.core.response.pprint_utils import pprint_response
response = query_engine.query("Give me a quick summary of the Andrew Huberman podcast with dr. Gina Poe.")
pprint_response(response, show_source=True)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.router_query_engine:Selecting query engine 0: This choice is relevant for summarizing the podcast content about sleep and memory with Dr. Gina Poe..
Selecting query engine 0: This choice is relevant for summarizing the podcast content about sleep and memory with Dr. Gina Poe..
Selecting query engine 0: This choice is relevant for summarizing the podcast content about sleep and memory with Dr. Gina Poe..
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/

In [33]:
response = query_engine.query("How does alcohol affect sleep?")
pprint_response(response, show_source=True)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.router_query_engine:Selecting query engine 1: Useful for retrieving specific content about sleep and memory in the podcast topic, which would include information on how alcohol affects sleep.
Selecting query engine 1: Useful for retrieving specific content about sleep and memory in the podcast topic, which would include information on how alcohol affects sleep.
Selecting query engine 1: Useful for retrieving specific content about sleep and memory in the podcast topic, which would include information on how alcohol affects sleep.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Re